# 首先要搞清楚该项目的几个阶段
- 1.pretrain阶段
- 2.finetune阶段
- 3.post-train阶段
每个阶段都有对应的数据集<br>
我们下载到../data文件夹中进行保存


## pretrain数据集说明
pretrain阶段主要用来让模型学会认字<br>
一般包含有大量的潜在语义信息<br>
我们打印其中的内容查看：




In [1]:
with open("../data/pretrain_hq.jsonl", "r", encoding="utf-8") as f:
    first_line = f.readline()  # 读取第一行
    print(first_line)

{"text": "<|im_start|>鉴别一组中文文章的风格和特点，例如官方、口语、文言等。需要提供样例文章才能准确鉴别不同的风格和特点。<|im_end|> <|im_start|>好的，现在帮我查一下今天的天气怎么样?今天的天气依据地区而异。请问你需要我帮你查询哪个地区的天气呢？<|im_end|> <|im_start|>打开闹钟功能，定一个明天早上七点的闹钟。好的，我已经帮您打开闹钟功能，闹钟将在明天早上七点准时响起。<|im_end|> <|im_start|>为以下场景写一句话描述：一个孤独的老人坐在公园长椅上看着远处。一位孤独的老人坐在公园长椅上凝视远方。<|im_end|> <|im_start|>非常感谢你的回答。请告诉我，这些数据是关于什么主题的？这些数据是关于不同年龄段的男女人口比例分布的。<|im_end|> <|im_start|>帮我想一个有趣的标题。这个挺有趣的：\"如何成为一名成功的魔术师\" 调皮的标题往往会吸引读者的注意力。<|im_end|> <|im_start|>回答一个问题，地球的半径是多少？地球的平均半径约为6371公里，这是地球自赤道到两极的距离的平均值。<|im_end|> <|im_start|>识别文本中的语气，并将其分类为喜悦、悲伤、惊异等。\n文本：“今天是我的生日！”这个文本的语气是喜悦。<|im_end|>"}



可以看到，其json内容为：
```{
    "text":
            "
    <|im_start|>鉴别一组中文文章的风格和特点，例如官方、口语、文言等。需要提供样例文章才能准确鉴别不同的风格和特点。
    <|im_end|> 
    <|im_start|>好的，现在帮我查一下今天的天气怎么样?今天的天气依据地区而异。请问你需要我帮你查询哪个地区的天气呢？
    <|im_end|> 
    <|im_start|>打开闹钟功能，定一个明天早上七点的闹钟。好的，我已经帮您打开闹钟功能，闹钟将在明天早上七点准时响起。
    <|im_end|> 
    <|im_start|>为以下场景写一句话描述：一个孤独的老人坐在公园长椅上看着远处。一位孤独的老人坐在公园长椅上凝视远方。
    <|im_end|> 
    <|im_start|>非常感谢你的回答。请告诉我，这些数据是关于什么主题的？这些数据是关于不同年龄段的男女人口比例分布的。
    <|im_end|> 
    <|im_start|>帮我想一个有趣的标题。这个挺有趣的：\"如何成为一名成功的魔术师\" 调皮的标题往往会吸引读者的注意力。
    <|im_end|> 
    <|im_start|>回答一个问题，地球的半径是多少？地球的平均半径约为6371公里，这是地球自赤道到两极的距离的平均值。
    <|im_end|> 
    <|im_start|>识别文本中的语气，并将其分类为喜悦、悲伤、惊异等。\n文本：“今天是我的生日！”这个文本的语气是喜悦。
    <|im_end|>
            "
}```
是这样子的一长串的对话文本，用<|im_start|>和<|im_end|>标记，和qwen等一致

## sft数据
sft主要是用于对齐model的语气，口吻等的数据集
打印数据结构：


In [1]:
with open("../data/sft_1024.jsonl", "r", encoding="utf-8") as f:
    first_line = f.readline()  # 读取第一行
    print(first_line)

{"conversations": [{"role": "user", "content": "What is the most recent version of the Android operating system?"}, {"role": "assistant", "content": "As of my last update in October 2023, the most recent version of the Android operating system is Android 14. It was officially released in August 2023. Android 14 brings several new features and improvements, including enhanced privacy controls, performance optimizations, and user interface refinements. For the most current information, you can check the official Android website or the latest announcements from Google."}]}



可以看到其数据结构为：
[```{"role": "user", "content": "What is the most recent version of the Android operating system?"}```,

 ```{"role": "assistant", "content": "As of my last update in October 2023, the most recent version of the Android operating system is Android 14. It was officially released in August 2023. Android 14 brings several new features and improvements, including enhanced privacy controls, performance optimizations, and user interface refinements. For the most current information, you can check the official Android website or the latest announcements from Google."}```]
一段对话。


## post-training数据

In [2]:
with open("../data/dpo.jsonl", "r", encoding="utf-8") as f:
    first_line = f.readline()  # 读取第一行
    print(first_line)

{"chosen": [{"content": "How many moles of HBr are required to react with 2 moles of C2H6 to form 2 moles of C2H5Br along with 2 moles of H2?", "role": "user"}, {"content": "To determine the number of moles of HBr required to react with 2 moles of C2H6 (ethane) to form 2 moles of C2H5Br (ethyl bromide) and 2 moles of H2, we need to write the balanced chemical equation for the reaction.\n\nThe reaction is a halogenation reaction, specifically the reaction of ethane with hydrogen bromide (HBr) to form ethyl bromide and hydrogen gas. The balanced chemical equation for this reaction is:\n\n\\[ \\text{C}_2\\text{H}_6 + \\text{HBr} \\rightarrow \\text{C}_2\\text{H}_5\\text{Br} + \\text{H}_2 \\]\n\nHowever, this equation is not balanced as written. To balance it, we need to ensure that the number of atoms of each element on the reactant side is equal to the number of atoms of that element on the product side. The balanced equation is:\n\n\\[ \\text{C}_2\\text{H}_6 + \\text{HBr} \\rightarrow \

内容为：```{
    "chosen": 

    [

    {"content": "How many moles of HBr are required to react with 2 moles of C2H6 to form 2 moles of C2H5Br along with 2 moles of H2?","role": "user"}, 

    {"content": "To determine the number of moles of HBr required to react with 2 moles of C2H6 (ethane) to form 2 moles of C2H5Br (ethyl bromide) and 2 moles of H2, we need to write the balanced chemical equation for the reaction.\n\nThe reaction is a halogenation reaction, specifically the reaction of ethane with hydrogen bromide (HBr) to form ethyl bromide and hydrogen gas. The balanced chemical equation for this reaction is:\n\n\\[ \\text{C}_2\\text{H}_6 + \\text{HBr} \\rightarrow \\text{C}_2\\text{H}_5\\text{Br} + \\text{H}_2 \\]\n\nHowever, this equation is not balanced as written. To balance it, we need to ensure that the number of atoms of each element on the reactant side is equal to the number of atoms of that element on the product side. The balanced equation is:\n\n\\[ \\text{C}_2\\text{H}_6 + \\text{HBr} \\rightarrow \\text{C}_2\\text{H}_5\\text{Br} + \\text{H}_2 \\]\n\nUpon closer inspection, we see that the equation is already balanced as written, with 1 mole of C2H6 reacting with 1 mole of HBr to produce 1 mole of C2H5Br and 1 mole of H2.\n\nGiven that you have 2 moles of C2H6, and the stoichiometry of the reaction is 1:1 for C2H6 to HBr, you would need 2 moles of HBr to react with 2 moles of C2H6 to form 2 moles of C2H5Br and 2 moles of H2.\n\nTherefore, the number of moles of HBr required is **2 moles**.", "role": "assistant"}
    ], 

    "rejected": [

        {

        "content": "How many moles of HBr are required to react with 2 moles of C2H6 to form 2 moles of C2H5Br along with 2 moles of H2?", "role": "user"}, 

        {"content": "To answer this question, we need to write down the chemical equation representing the reaction between hydrogen bromide (HBr) and ethane (C2H6):\n\nC2H6(g) + 2HBr(g) \u2192 C2H5Br(l) + H2(g)\n\nNow, we can set up a mole ratio based on the coefficients in front of each substance in the balanced equation:\n\nMoles of HBr : Moles of C2H6 :: 2 : 1\n\nWe also know that there are two moles of C2H6 present in our starting mixture, so we can substitute this value into the above mole ratio to determine the amount of HBr needed:\n\n(2 mol C2H6)/(1 mol C2H6) x 2 mol HBr = 4 mol HBr\n\nSo, four moles of HBr would be required to completely react with two moles of C2H6 according to the given stoichiometric relationship.", "role": "assistant"}
        
        ]
}```

## 制作Dataset加载器


### Pretrain阶段Dataset


In [ ]:
import torch
import json
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer
class PretrainDataset(Dataset):
    def __init__(self,data_path,tokenizer,max_seqlen=1024):
        self.data_path = data_path
        self.tokenizer = tokenizer
        self.max_seqlen = max_seqlen
        self.samples = self.load_data(data_path)
    
    def load_data(self, path):
        samples = []
        with open(path, 'r', encoding='utf-8') as f:
            for line_num, line in enumerate(f, 1):
                data = json.loads(line.strip())
                samples.append(data)
        return samples
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, index):
        sample = self.samples[index]
        text = str(sample['text'])
        encoding = self.tokenizer(
            text,
            max_length=self.max_seqlen,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        input_ids = encoding.input_ids.squeeze()   
        loss_mask = (input_ids != self.tokenizer.pad_token_id) # 确认input_ids中非填充部分，loss_mask形状类似[True, True, ..., False]
        X=torch.tensor(input_ids[:-1],dtype=torch.long)  # 去掉最后一个token
        Y=torch.tensor(input_ids[1:],dtype=torch.long)  # 去掉第一个
        loss_mask = torch.tensor(loss_mask[1:], dtype=torch.long)
        return X, Y, loss_mask



